In [ ]:
import torch
import pandas as pd
import json
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import random
import numpy as np


device = "cuda" if torch.cuda.is_available() else "cpu"


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)  


text_embedding_model = SentenceTransformer("/root/autodl-tmp/all-MiniLM-L6-v2")


def load_text_retrieval_system(index_file, texts_file):
    index = faiss.read_index(index_file)
    with open(texts_file, "r", encoding="utf-8") as f:
        texts = json.load(f)
    return index, texts


def retrieve_best_text(query, index, texts):
    query_embedding = text_embedding_model.encode([query], convert_to_tensor=False)
    distances, indices = index.search(query_embedding.astype("float32"), 1)

    best_index = indices[0][0]
    best_text = texts[best_index]
    best_similarity = 1 / (1 + distances[0][0]) 
    
    return best_text if best_similarity > 0.025 else ""


def load_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)
    model = AutoModelForCausalLM.from_pretrained(
        model_path,
        torch_dtype=torch.bfloat16,
        low_cpu_mem_usage=True,
        trust_remote_code=True
    ).to(device).eval()
    return model, tokenizer


def format_query(user_input, retrieved_text):
    context = f"Retrieved Document: {retrieved_text}" if retrieved_text else ""
    return f"### User Question ###\n{user_input}\nBased on the symptoms I described, what disease do you think I have? Please give the disease name before proceeding with the subsequent analysis.\n### Retrieved Context ###\n{context}\n\n### Assistant Answer ###\n"


def generate_response(model, tokenizer, query):
    inputs = tokenizer(query, return_tensors="pt").to(device)
    gen_kwargs = {"max_new_tokens": 50, "do_sample": True, "temperature": 0.7, "top_k": 50}
    with torch.no_grad():
        outputs = model.generate(**inputs, **gen_kwargs)
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response.replace(query, "").strip()


def process_excel_with_rag(file_path, output_path, model_path, index_file, texts_file):
    df = pd.read_excel(file_path)
    
    if "Response after RAG" not in df.columns:
        df["Response after RAG"] = ""
    df["Response after RAG"] = df["Response after RAG"].astype(str)
    
    model, tokenizer = load_model(model_path)
    index, texts = load_text_retrieval_system(index_file, texts_file)
    
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing with RAG"):
        symptom_text = row["Symptoms"]
        retrieved_text = retrieve_best_text(symptom_text, index, texts)
        query_with_rag = format_query(symptom_text, retrieved_text)
        
        response = generate_response(model, tokenizer, query_with_rag)
        df.at[i, "Response after RAG"] = str(response)
        
        if i < 20:
            print(f"--- Record {i+1} ---")
            print(f"Symptoms: {symptom_text}")
            # print(f"Retrieved Text: {retrieved_text}")
            print(f"Generated Response: {response}")
            print("\n" + "="*80 + "\n")
    
    df.to_excel(output_path, index=False)


if __name__ == "__main__":
    input_file = "/root/autodl-tmp/nhs_test.xlsx"
    output_file = "/root/autodl-tmp/nhs_test_with_rag.xlsx"
    model_path = "/root/autodl-tmp/trained_model_MedQA"
    index_file = "/root/autodl-tmp/NHS_text_index.faiss"
    texts_file = "/root/autodl-tmp/NHS_text_texts.json"
    
    process_excel_with_rag(input_file, output_file, model_path, index_file, texts_file)


Processing with RAG:   0%|          | 1/410 [00:03<24:23,  3.58s/it]

--- Record 1 ---
Symptoms: In most cases, this condition causes no noticeable symptoms. However, if it becomes large, some people may develop pain or a pulsating feeling in their abdomen (tummy) or persistent back pain. This condition doesn’t usually pose a serious threat to health, but there is a risk that a larger abnormality could rupture. A rupture can cause massive internal bleeding, which is usually fatal. Around 8 out of 10 people with a rupture either die before they reach the hospital or do not survive surgery. The most common symptom of a rupture is sudden and severe abdominal pain. If you suspect that you or someone else has had a rupture, call emergency services immediately and ask for an ambulance.
Generated Response: Aortic aneurysm

### Step 1: Identify the disease from the symptoms described
The disease described is an aortic aneurysm, which is characterized by the dilation of the aorta. The symptoms described are a pulsating




Processing with RAG:   0%|          | 2/410 [00:06<21:44,  3.20s/it]

--- Record 2 ---
Symptoms: Symptoms may vary from person to person and can appear gradually or suddenly. Common symptoms include swelling, pain that worsens during or after movement or exercise, stiffness that is more pronounced in the morning or after periods of rest, tenderness when touching the affected area, mild heat, and a loss of movement and strength around the ankle and foot.
Generated Response: Osteoarthritis of the knee

### Step 1: Identify the symptoms described
The symptoms described include swelling, pain that worsens during or after movement or exercise, stiffness that is more pronounced in the morning or after periods of rest,




Processing with RAG:   1%|          | 3/410 [00:09<20:55,  3.09s/it]

...

Processing with RAG:   5%|▍         | 19/410 [00:56<18:44,  2.88s/it]

--- Record 19 ---
Symptoms: Even after receiving adrenaline, individuals experiencing this condition should be taken to a hospital for observation, typically for 6 to 12 hours, as symptoms may occasionally reappear. During hospitalization, oxygen therapy may be provided to assist with breathing, and intravenous fluids may be administered to help stabilize blood pressure. In addition to adrenaline, other medications such as antihistamines and corticosteroids may be used to alleviate symptoms. Blood tests might also be conducted to confirm the condition. Discharge from the hospital is typically possible once symptoms are under control and unlikely to return quickly, though severe cases may require a longer stay. Patients may be advised to continue taking antihistamines and corticosteroids for 2 to 3 days after leaving the hospital to prevent recurrence.
Generated Response: Chronic pain

### Step 1: Identify key features of the condition described in the question.
The condition described 

Processing with RAG:   5%|▍         | 20/410 [00:59<18:41,  2.88s/it]

--- Record 20 ---
Symptoms: The primary symptom of this condition is chest pain, which may present as a dull ache, a heavy sensation, or a feeling of tightness in the chest. The pain can also spread to the arms, neck, jaw, or back and is often triggered by physical exertion. While this condition itself is not life-threatening, prolonged chest pain could indicate a more serious event, such as a heart attack.
Generated Response: Angina

### Step 1: Determine the primary symptom of the condition
The primary symptom of the condition is chest pain, which may present as a dull ache, a heavy sensation, or a feeling of tightness in the chest.

### Step




Processing with RAG: 100%|██████████| 410/410 [19:54<00:00,  2.91s/it]
